# Earth Engine Reducer Concept

In [1]:
import ee
ee.Initialize()
ee.Authenticate()

True

Basic

In [2]:
my_list = ee.List.sequence(1, 10)
mean = my_list.reduce(ee.Reducer.mean())

print(f"ee.List: {my_list.getInfo()}")
print(f"Mean: {mean.getInfo()}")

ee.List: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Mean: 5.5


Reduce ee.ImageCollection

Applies a reducer to all of the bands of an image. It will called at each pixel to reduce the stack of band values. The output image will have one band for each reducer output. Output an ee.Image.

In [6]:
import geemap

s2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
geometry = ee.Geometry.BBox(106.7537, -6.2624, 106.8992, -6.12049) # Jakarta

s2 = (
    s2
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))
    .filter(ee.Filter.date('2019-01-01', '2020-01-01'))
    .filter(ee.Filter.bounds(geometry))
    .select('B.*')
)

image = s2.reduce(ee.Reducer.mean()) # ee.Image

print(f"Dtype: {type(image)}")
print(f"ImageCollection size: {s2.size().getInfo()}")

Map = geemap.Map()
rgb_vis = {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']}
rgb_vis_mean = {'min': 0, 'max': 3000, 'bands': ['B4_mean', 'B3_mean', 'B2_mean']}
Map.addLayer(s2.first().clip(geometry), rgb_vis, 'Sentinel-2')
Map.addLayer(image.clip(geometry), rgb_vis_mean, 'Sentinel-2 Mean')
Map.centerObject(geometry)
Map

Dtype: <class 'ee.image.Image'>
ImageCollection size: 60


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

Calculate average value in each band.

Apply a reducer to all the pixels in a specific region. Returns an ee.dictionary.Dictionary of the reducer's output.

In [17]:
stats = image.clip(geometry).reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=geometry,
    scale=10,
    maxPixels=1e10,
    tileScale=4,
)
print(stats.getInfo())
print(stats.get('B4_mean').getInfo())

{'B10_mean': 27.744499399922233, 'B11_mean': 2344.079549701919, 'B12_mean': 1814.704796322894, 'B1_mean': 2472.120784821824, 'B2_mean': 2242.574905610044, 'B3_mean': 2046.0149182998648, 'B4_mean': 2040.7083500040692, 'B5_mean': 2004.2813631785023, 'B6_mean': 2268.9151926216296, 'B7_mean': 2483.4602959071885, 'B8A_mean': 2570.5994356245737, 'B8_mean': 2259.390739839758, 'B9_mean': 476.91329125576385}
2040.7083500040692
